# NIH Chest X-Ray: A Multi-Label Classification Problem

## Masalah apa yang ingin dipecahkan?
Untuk mengidentifikasi berbagai jenis penyakit paru

## Jenis penyakit apa saja yang diteliti?
Ringkasan dari penyakit yang akan kami teliti:

| Penyakit     | Deskripsi | Ciri-ciri |
| ---        |    ----   | --- |
| Atelectasis |Kerusakan total atau sebagian dari seluruh paru-paru atau area (lobus) paru-paru. Ini terjadi ketika kantung udara kecil (alveoli) di dalam paru-paru mengempis atau mungkin diisi dengan cairan alveolar | Sulit bernafas, pernafasan cepat dan dangkal, batuk |
| Cardiomegaly | Kondisi ketika jantung mengalami pembesaran akibat penyakit tertentu | Pada beberapa kasus, kondisi ini bisa terjadi tanpa menimbulkan gejala. Namun, ada juga kardiomegali yang menyebabkan pusing, lemas, dan sesak napas |
| Consolidation | Penyakit paru-paru yang terjadi ketika udara yang biasanya mengisi saluran udara kecil di paru-paru diganti dengan sesuatu yang lain (seperti nanah atau darah) | Batuk dengan dahak kental berwarna hijau, batuk darah, nyeri dada, demam |
| Effusion | Penumpukan cairan berlebih di antara lapisan pleura di luar paru-paru   | Nyeri dada, batuk kering, dispnea (sesak napas), orthopnea (ketidakmampuan bernapas dengan mudah kecuali orang tersebut duduk atau berdiri tegak |
| Hernia | Cacat kongenital yang terdapat sebuah lubang pada otot diafragma (otot yang membatasi rongga dada dan rongga perut). Lubang pada otot diafragma memberikan kesempatan organ-organ pada rongga perut seperti usus dapat naik masuk ke rongga dada | Kebiruan pada kulit karena kekurangan oksigen, denyut jantung yang cepat, laju pernapasan yang cepat |
| Infiltration | Terjadi ketika zat yang lebih padat daripada udara (misalnya, nanah, edema, darah, surfaktan, protein, atau sel) tetap berada di dalam parenkim paru   | Demam, berkeringat malam, berat badan turun, dan mudah lelah |
| Mass | Pembelahan sel atau kematian sel abnormal di jaringan paru-paru atau di saluran udara yang mengarah ke paru-paru, biasa disebut dengan kanker paru-paru | Batuk yang tidak kunjung hilang, batuk darah atau dahak berwarna karat, nyeri dada, suara serak, kehilangan selera makan, penurunan berat badan, sesak napas, merasa lelah dan lemas |
| Nodule  | Pertumbuhan abnormal yang terbentuk di paru-paru yang biasanya tidak bersifat kanker  | Nyeri dada, batuk kronis atau batuk darah, suara serak, infeksi pernapasan berulang seperti bronchitis atau pneumonia, sesak napas |
| Pneumonia | Pertumbuhan abnormal yang terbentuk di paru-paru yang biasanya tidak bersifat kanker | Nyeri dada, batuk kronis atau batuk darah, suara serak, infeksi pernapasan berulang seperti bronchitis atau pneumonia, sesak napas |
| Pneumothorax | Penyakit yang disebabkan oleh cedera dada tumpul atau tembus, prosedur medis tertentu, atau kerusakan akibat penyakit paru-paru yang mendasarinya| Nyeri dada, sesak napas, kulit kebiruan akibat kurang oksigen, napas dan detak jantung cepat, batuk kering |

## Datanya diambil dari mana?
Dataset ini diambil dari [https://nihcc.app.box.com/v/ChestXray-NIHCC](https://nihcc.app.box.com/v/ChestXray-NIHCC)

## 0. Setup

Menggunakan GPU dari Kaggle untuk mempercepat riset karena data yang digunakan kurang lebih ~45GB...

In [ ]:
!nvidia-smi -L

Beberapa library yang akan digunakan untuk kepentingan riset ini...

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(rc={'figure.figsize':(10, 7)}, style='darkgrid')
sns.set_color_codes()

from scipy.stats import norm

import os

import glob
import datetime
import itertools
import random

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

# %config Completer.use_jedi = False # intellisense

## 1. Ambil data + Data Cleaning

Langkah pertama untuk memulai riset ini yaitu untuk mengambil data lalu import data.

In [ ]:
data = pd.read_csv("../input/data/Data_Entry_2017.csv")
print(f"Terdapat {data.shape[0]} jumlah data pada dataset NIH Chest X-Rays")
print(f"Terdapat {data.shape[1]} kolom pada dataset NIH Chest X-Rays")
data.head()

| Kolom      | Deskripsi |
| ---        |    ----   |
| `Image Index` | Base path image yang nantinya akan kami gunakan untuk mencari path dari folder gambar yang telah diimport|
| `Finding Labels` | Jenis penyakit dari setiap gambar (beberapa terdapat lebih dari 1 penyakit per gambar (multi-label)|
| `Follow-up #` | -       |
| `Patient ID` | ID unik pasien yang nantinya dapat digunakan untuk analisis lebih lanjut pada bab berikut (melihat jenis kelamin unik)|
| `Patient Age` | Umur pasien yang diteliti|
| `Patient Gender` | Jenis kelamin pasien (`M`: Pria, `F`: Wanita)|
| `View Position` | `PA` / `Posterior-Anterior (PA)`: Gambar rontgen dada yang dihasilkan terlihat seolah-olah melihat pasien dari depan, tatap muka. Jantung berada di sisi kanan gambar saat dilihat. `AP` / `Anterior-Posterior (AP)`: Kadang-kadang tidak mungkin bagi radiografer untuk mendapatkan rontgen dada PA. Ini biasanya karena pasien terlalu sakit untuk berdiri. Gambar rontgen dada masih terlihat seolah-olah menatap pasien secara langsung |
| `OriginalImage[Width`     | Size lebar gambar asli|
| `Height]` | Size tinggi gambar asli|
| `OriginalImagePixelSpacing[x` | Size lebar gambar dalam bentuk `normalization` yang memiliki nilai [0-1] |
| `y]`     | Size tinggi gambar dalam bentuk `normalization` yang memiliki nilai [0-1] |
| `Unnamed: 11`     | -       |

### Menghilangkan beberapa jenis penyakit

Berikutnya kami akan mencari label penyakit unik pada dataset:

1. Setiap label penyakit per data akan dilakukan `.split('|')` yang nantinya akan menghasilkan sebuah array yang berisi jenis penyakit per data 
2. Menggunakan `itertools.chain` yang mengambil array jenis penyakit dari hasil `.split('|')` menjadi array 1 dimensi
 - Data yang sudah di-split akan di terapkan `starred operator` atau gampangnya disebut `rest operator` untuk menyalin isi data tersebut sebagai argumen pada `itertools.chain`
 - Berikutnya `itertools.chain(["a", "b"], ["c", "d"]) --> a b c d`
3. Menggunakan `np.unique` untuk mengambil data uniknya dari array `itertools.chain`

In [ ]:
# Mencari label penyakit unik
labels = list(np.unique(list(itertools.chain(*data['Finding Labels'].map(lambda x: x.split('|'))))))
print(f"Labels: {labels}")
print(f"Unique labels: {len(labels)}")

In [ ]:
# Menghapus beberapa jenis penyakit (tersisa 10 jenis penyakit)
to_remove = {"Pleural_Thickening", "Fibrosis", "Edema", "Emphysema"}
labels = [x for x in labels if x not in to_remove]

print(f"Before removing: {data.shape[0]} images")
for i in range(len(data)):
    disease = data['Finding Labels'][i].split("|")
    for j in disease:
        if j in to_remove:
            data.drop(i, inplace=True)
            break
print(f"After removing: {data.shape[0]} images")
data.index = np.arange(0, data.shape[0]) # reset index

In [ ]:
labels = [x for x in labels if x != 'No Finding']
labels

## 2. Eksplorasi Data

In [ ]:
plt.figure(figsize=(30, 10))
finding_labels = pd.Series(data['Finding Labels'].value_counts(ascending=False)[:30])
ax = sns.barplot(y=finding_labels.index, x=finding_labels.values, palette="Paired")
for container in ax.containers:
    ax.bar_label(container)
plt.suptitle("Jumlah penyakit per label", fontsize=16, fontweight="bold", y="0.92");

Terlihat disini bahwa ada beberapa label terletak di satu data maka tepat disini bahwa ini merupakan masalah klasifikasi multi-label.

Dataset ini dapat disimpulkan bahwa terdapat `imbalanced` data.

In [ ]:
# Meghitung jenis kelamin pasien
data = data[data['Patient Age'] <= 100]
data_unique_id = data.drop_duplicates(subset=['Patient ID'])

# Plot
plt.figure(figsize=(15, 5))
ax = sns.barplot(y=data_unique_id['Patient Gender'].value_counts().index, x=data_unique_id['Patient Gender'].value_counts().values, palette="Paired");
for container in ax.containers:
    ax.bar_label(container)
plt.title("Jumlah pasien unik berdasarkan jenis kelamin", fontsize=16, fontweight="bold");

Pasien unik untuk pria yaitu sekitar 16 ribu pasien serta 14 ribu pasien untuk yang wanita.

In [ ]:
# Melihat distribusi umur
ages = data_unique_id['Patient Age']
bins = np.arange(0, 101, 10)

# Memuat rentang umur
cats = pd.Series(pd.cut(ages, bins).value_counts())

# Plot
plt.figure(figsize=(30, 7))
plt.subplot(121)
ax = sns.barplot(x=cats.values, y=cats.index, palette="flare")
for container in ax.containers:
    ax.bar_label(container)
plt.xlabel("Age count")
plt.ylabel("Age range")
    
# Plot distplot
plt.subplot(122)
ax = sns.distplot(data_unique_id['Patient Age'].values, color="m")
plt.xlabel("Age")

plt.suptitle("Distribusi umur", fontsize=24, fontweight="bold");

Disini terlihat bahwa mayoritas umur pasien yang diamati yaitu sekitar umur ~20-70 tahun.

In [ ]:
# Distribusi umur berdasarkan jenis kelamin
data_unique_id['Age Range'] = pd.cut(ages, bins)

ax = sns.catplot(x="Age Range", col="Patient Gender", data=data_unique_id, kind="count", aspect=0.9, size=8, palette="flare")
ax.set_xticklabels(['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100'])
ax.fig.suptitle('Distribusi umur berdasarkan jenis kelamin', fontsize=20, fontweight="bold")
ax.fig.subplots_adjust(top=0.9);

### Mencari path gambar dari setiap Image Index

Langkah yang akan dilakukan untuk mencari path gambar serta melakukan mapping dengan Image Index:
1. Load seluruh path gambar dari folder `../input/data/images*/images/*.png` (`*`: mensimbolkan untuk melihat seluruh file/folder untuk direktori tersebut)

In [ ]:
# Mencari path gambar dgn format {"image001.png": "../input/data/images_001/images/image001.png"}
seluruh_path_gambar = glob.glob('../input/data/images*/images/*.png')
path_gambar_dict = {os.path.basename(x): x for x in seluruh_path_gambar}

2. Memuat dictionary untuk key dan value pair
3. Melihat `*.png` sebagai key (menggunakan `os.path.basename` untuk melakukan indexing pada dictionary agar dapat mengambil value untuk index tersebut) dan memuat kolom `path_gambar` sebagai value dari key tersebut lalu menyimpan value dari key/Image Index pada kolom `path_gambar`

In [ ]:
# Key, value dari path_gambar akan dimap dengan Image Index lalu menyimpan value tersebut di kolom path_gambar
data['path_gambar'] = data['Image Index'].map(path_gambar_dict.get)

Mantap! 

Melihat datasetnya sekali lagi...

In [ ]:
data.sample(3)

Sudah terlihat seperti yang diinginkan...!

### Visualisasi sebuah gambar

Hal terpenting dalam melakukan riset yaitu melihat isi dari datanya, maupun teks atau gambar ataupun lain-lain, setidaknya kita harus bisa mengenal lebih lanjut dari data yang kami bekerja dengan

Maka dari itu, kami akan menampilkan sebuah/beberapa gambar dari dataset

Disini kami akan menvisualisasikan sebuah gambar terlebih dahulu, lalu nantinya akan kami visualisasikan beberapa gambar random

In [ ]:
from PIL import Image

fig, ax = plt.subplots()

# Melihat gambar dari data ke-2 dari atas
img = data.iloc[:, -1][2]
label = data.iloc[:, 1][2]

arr = Image.open(img) # load image
ax.imshow(arr, cmap="bone")
ax.set_title(label)
ax.axis(False);

Diatas adalah contoh gambar yang akan kami teliti lebih lanjut

#### Visualisasi beberapa gambar random

Setelah menvisualisasikan sebuah gambar, sekarang kami sudah mendapatkan ide untuk menvisualisasikan beberapa gambar random dari dataset

In [ ]:
import random
from PIL import Image

# Plot gambar random dari dataset
def plot_random_image(k=16):
    rnd_sample_idx = random.sample(range(data.shape[0]), k=k)
    
    # Plot
    plt.figure(figsize=(20, 20))
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
    
    for i, idx in enumerate(rnd_sample_idx):
        plt.subplot(4, 4, i + 1)
        img = data.iloc[:, -1][idx]
        label = data.iloc[:, 1][idx]

        arr = Image.open(img) # load image
        plt.imshow(arr, cmap="bone")
        plt.title(label)
        plt.axis(False)
    
    plt.suptitle("Beberapa gambar dari dataset", fontsize=16, fontweight="bold", y="0.91")
    plt.show()

In [ ]:
plot_random_image()

#### Visualisasi gambar dengan `bbox`

Dari dataset yang diteliti terdapat dataset untuk bounding box untuk beberapa gambar saja, namun, karena riset ini lebih kepada klasifikasi, kami akan gunakan dataset bounding box tersebut untuk visualisasi saja

Pertama-tama, load dataset bounding box terlebih dahulu

In [ ]:
data_bbox = pd.read_csv("../input/data/BBox_List_2017.csv")
print(f"Terdapat {data_bbox.shape[0]} jumlah data pada dataset NIH Chest X-Rays (dengan penyakit)")
print(f"Terdapat {data_bbox.shape[1]} kolom pada dataset NIH Chest X-Rays (dengan penyakit)")
data_bbox.head(3)

In [ ]:
label_counts_data_bbox = data_bbox["Finding Label"].value_counts()

plt.figure(figsize=(30, 5))
plt.subplot(121)
ax = sns.barplot(x=label_counts_data_bbox.values, y=label_counts_data_bbox.index, palette="flare")

for container in ax.containers:
    ax.bar_label(container)
    
plt.xlabel("Jumlah")
plt.ylabel("Penyakit")
plt.title("Jumlah per label penyakit dari dataset dengan bounding box", fontsize=16, fontweight="bold");

Menggunakan algoritma pencari path gambar dengan Image Index (sudah ada diatas)

In [ ]:
# Key dari path_gambar akan dimap dengan Image Index lalu menyimpan value tersebut di kolom path gambar
data_bbox['path_gambar'] = data_bbox['Image Index'].map(path_gambar_dict.get)
data_bbox.sample(3)

Path gambar yang sudah ditemukan untuk setiap Image Index pada dataset bounding box kemudian akan digunakan untuk visualisasi gambar random

Pertama-tama, kami akan menvisualisasikan sebuah gambar dari dataset bounding box

In [ ]:
from PIL import Image
import matplotlib.patches as patches # untuk plot kotak

fig, ax = plt.subplots()

img = data_bbox.iloc[:, -1][2]
label = data_bbox.iloc[:, 1][2]

arr = Image.open(img)
ax.imshow(arr, cmap='bone')

x = data_bbox['Bbox [x'][2]
y = data_bbox['y'][2]
w = data_bbox['w'][2]
h = data_bbox['h]'][2]

# Memuat kotak
rect = patches.Rectangle((x, y), w, h, linewidth=5, edgecolor='r', facecolor='none')

# Menambahkan kotak ke gambar
ax.add_patch(rect)

ax.set_title(label)
ax.axis(False);

Setelah kami mendapatkan ide untuk menvisualisasikan sebuah gambar dari dataset yang memiliki bounding box, berikutnya kami dapat menvisualisasikan beberapa gambar random dari dataset tersebut

### Visualisasi setiap jenis penyakit

In [ ]:
import random
from PIL import Image
import matplotlib.patches as patches # gambar kotak

# Plot setiap gambar dengan label penyakit unik
def plot_unique_labels_with_bbox(k=8):
    unique_labels = set() # set
    unique_idx = []
    for i in range(data_bbox.shape[0]):
        if data_bbox["Finding Label"][i] not in unique_labels:
            unique_idx.append(i)
            unique_labels.add(data_bbox["Finding Label"][i])
    
    # Plot
    plt.figure(figsize=(20, 20))
#     plt.subplots_adjust(hspace=0.5)
    
    for i, idx in enumerate(unique_idx):
        ax = plt.subplot(3, 3, i + 1)
        img = data_bbox.iloc[:, -1][idx]
        label = data_bbox.iloc[:, 1][idx]

        arr = Image.open(img)
        ax.imshow(arr, cmap="bone")
        
        x = data_bbox['Bbox [x'][idx]
        y = data_bbox['y'][idx]
        w = data_bbox['w'][idx]
        h = data_bbox['h]'][idx]
        
        # Memuat kotak
        rect = patches.Rectangle((x, y), w, h, linewidth=5, edgecolor='r', facecolor='none')

        # Menambahkan kotak ke gambar
        ax.add_patch(rect)
        
        ax.set_title(label)
        ax.axis(False)

    plt.suptitle("Macam-macam penyakit", fontsize=20, fontweight="bold", y="0.91", x="0.5")
    plt.show()

In [ ]:
plot_unique_labels_with_bbox()

### Visualisasi gambar random dengan bounding box

In [ ]:
import random
from PIL import Image
import matplotlib.patches as patches # gambar kotak

# Plot gambar random dari dataset
def plot_random_image_with_bbox(k=16):
    rnd_sample_idx = random.sample(range(data_bbox.shape[0]), k=k)
    
    # Plot
    fig, axs = plt.subplots(4, 4, figsize=(20, 20))
    fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
    
    for (idx, ax) in zip(rnd_sample_idx, axs.flatten()):
        img = data_bbox.iloc[:, -1][idx]
        label = data_bbox.iloc[:, 1][idx]

        arr = Image.open(img)
        ax.imshow(arr, cmap="bone")
        
        x = data_bbox['Bbox [x'][idx]
        y = data_bbox['y'][idx]
        w = data_bbox['w'][idx]
        h = data_bbox['h]'][idx]
        
        # Memuat kotak
        rect = patches.Rectangle((x, y), w, h, linewidth=5, edgecolor='r', facecolor='none')

        # Menambahkan kotak ke gambar
        ax.add_patch(rect)
        
        ax.set_title(label)
        ax.axis(False)

    fig.suptitle("Beberapa penyakit + bbox", fontsize=20, fontweight="bold", y="1.03")
    fig.show()

In [ ]:
# Menvisualisasikan beberapa gambar random
plot_random_image_with_bbox()

Diatas adalah beberapa gambar random yang sudah dilengkapi dengan bounding box, namun ini tidak berlaku untuk setiap dataset utama, bounding box ini hanya terdapat pada beberapa gambar saja yang kurang lebih ada ~1000 jumlahnya

## 3. Data Cleaning

Beberapa kolom yang **TIDAK** akan kami gunakan yaitu untuk riset lebih lanjut yaitu:

| Kolom yang TIDAK akan digunakan     | Deskripsi |
| ---        |    ----   |
| `Image Index` | Base path image yang nantinya akan kami gunakan untuk mencari path dari folder gambar yang telah diimport|
| `Follow-up #` | -       |
| `Patient ID` | ID unik pasien yang nantinya dapat digunakan untuk analisis lebih lanjut pada bab berikut (melihat jenis kelamin unik)|
| `Patient Age` | Umur pasien yang diteliti|
| `Patient Gender` | Jenis kelamin pasien (`M`: Pria, `F`: Wanita)|
| `View Position` | `PA` / `Posterior-Anterior (PA)`: Gambar rontgen dada yang dihasilkan terlihat seolah-olah melihat pasien dari depan, tatap muka. Jantung berada di sisi kanan gambar saat dilihat. `AP` / `Anterior-Posterior (AP)`: Kadang-kadang tidak mungkin bagi radiografer untuk mendapatkan rontgen dada PA. Ini biasanya karena pasien terlalu sakit untuk berdiri. Gambar rontgen dada masih terlihat seolah-olah menatap pasien secara langsung |
| `OriginalImage[Width`     | Size lebar gambar asli|
| `Height]` | Size tinggi gambar asli|
| `OriginalImagePixelSpacing[x` | Size lebar gambar dalam bentuk `normalization` yang memiliki nilai [0-1] |
| `y]`     | Size tinggi gambar dalam bentuk `normalization` yang memiliki nilai [0-1] |
| `Unnamed: 11`     | -       |

In [ ]:
# Menyimpan kolom Finding Labels dan path_gambar
data = data[['Finding Labels', 'path_gambar']]
data

Berikutnya, menerapkan `one-hot encoding` pada setiap kolom dataset untuk mengindikasikan bahwa terdapat/tidak terdapatnya penyakit tersebut pada data tersetbu

In [ ]:
# Memuat 1 atau 0 / one-hot encoding pada setiap kolom label penyakit dari Finding Labels
for label in labels:
    data[label] = data['Finding Labels'].map(lambda exist: 1 if label in exist else 0)

In [ ]:
data.index = np.arange(0, len(data)) # reset index

# Melihat isi data
data.head()

## 4. Persiapan training dan testing

Data yang sudah dibersihkan kemudian akan dipersiapkan untuk training dan testing yang nantinya akan digunakan untuk melatih beberapa arstitektur computer vision

In [ ]:
# Memisah dataset utama menjadi train and test
train, test = train_test_split(data, train_size=0.8, random_state=42)
print(f"Terdapat {train.shape[0]} gambar untuk training\nTerdapat {test.shape[0]} gambar untuk testing")

In [ ]:
# Untuk dataset training dipecah menjadi training dan validation
X_train, X_test = train_test_split(train, train_size=0.8, random_state=42)
print(f"Terdapat {X_train.shape[0]} gambar untuk X_train\nTerdapat {X_test.shape[0]} gambar untuk X_test")

Dataset yang sudah dipecah menjadi train, validation dan test set kemudian diterapkan `ImageDataGenerator` untuk melakukan rescaling

In [ ]:
# Menggunakan ImageDataGenerator untuk melakukan transformasi gambar
train_datagen = ImageDataGenerator(rescale=1/255.)
valid_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

`ImageDataGenerator` dapat load gambar dari dataframe menggunakan instansinya yaitu `.flow_from_dataframe()` yang dimana nantinya gambar akan otomatis terrescale serta batching

Kami menerapkan setiap ukuran gambar menjadi `(224, 224)` serta 32 untuk `batch_size`

In [ ]:
# Memuat dataloader menggunakan flow_from_dataframe karena data kami terdapat di dataframe 
IMG_SHAPE = (224, 224)
BATCH_SIZE = 32

print("Training images:")
train_data = train_datagen.flow_from_dataframe(X_train,
                                             x_col="path_gambar",
                                             y_col=labels,
                                             target_size=IMG_SHAPE,
                                             classes=labels,
                                             color_mode="rgb",
                                             class_mode="raw",
                                             seed=42,
                                             batch_size=BATCH_SIZE)
print("Validation images:")
valid_data = valid_datagen.flow_from_dataframe(X_test,
                                             x_col="path_gambar",
                                             y_col=labels,
                                             target_size=IMG_SHAPE,
                                             classes=labels,
                                             color_mode="rgb",
                                             class_mode="raw",
                                             seed=42,
                                             batch_size=BATCH_SIZE)
print("Testing images:")
test_data = test_datagen.flow_from_dataframe(test,
                                             x_col="path_gambar",
                                             y_col=labels,
                                             target_size=IMG_SHAPE,
                                             classes=labels,
                                             color_mode="rgb",
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=42,
                                             batch_size=BATCH_SIZE)

Sekali lagi melihat dataset yang sudah dibatching...

In [ ]:
# Melihat isi batch size pertama dari training data
train_x, train_y = next(train_data)
print(f"Image dimensions: {train_x[1].shape}")
print(f"Diseases vector: {train_y[1]}")

## 5. Modeling

Setelah melewati data cleaning dan memecahkan data menjadi training, validation, dan testing, berikutnya, kami akan menggunakan model-model computer vision yang sudah siap untuk dilatih pada data yang telah disiapkan

Beberapa model yang kami gunakan untuk meneliti:
- VGG19
- DenseNet121
- InceptionV3
- ResNet50V2
- NASNet
- GoogleNet
- EfficientNet

### VGG19

In [ ]:
# from tensorflow.keras.layers import Input
# from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# # 1. Create model
# input_shape=(224, 224, 3)
# img_input = Input(shape=input_shape)

# base_model = VGG19(include_top=False, input_tensor=img_input, input_shape=input_shape, 
#                          pooling="avg", weights='imagenet')
# x = base_model.output
# predictions = Dense(len(labels), activation="sigmoid", name="output")(x)
# model = Model(inputs=img_input, outputs=predictions)


# # 2. Compile the model
# optimizer = Adam(lr=0.0005)
# model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()])

# earlystopper = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=3, verbose=0, mode='min',
#     restore_best_weights=True
# )

# # 3. Fit the model
# history = model.fit(train_data,
#                    epochs=5,
#                    steps_per_epoch=len(train_data),
#                    validation_data=valid_data,
#                    validation_steps=len(valid_data),
#                    callbacks=[earlystopper])

In [ ]:
# # Save the model
# model.save('vgg19.h5')

### NASNetMobile

In [ ]:
# from tensorflow.keras.layers import Input
# from tensorflow.keras.applications.nasnet import NASNetMobile
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# # 1. Create model
# input_shape=(224, 224, 3)
# img_input = Input(shape=input_shape)

# base_model = NASNetMobile(include_top=False, input_tensor=img_input, input_shape=input_shape, 
#                          pooling="avg", weights='imagenet')
# x = base_model.output
# predictions = Dense(len(labels), activation="sigmoid", name="output")(x)
# model = Model(inputs=img_input, outputs=predictions)


# # 2. Compile the model
# optimizer = Adam(lr=0.0005)
# model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()])

# earlystopper = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=3, verbose=0, mode='min',
#     restore_best_weights=True
# )

# # 3. Fit the model
# history = model.fit(train_data,
#                    epochs=5,
#                    steps_per_epoch=len(train_data),
#                    validation_data=valid_data,
#                    validation_steps=len(valid_data),
#                    callbacks=[earlystopper])

In [ ]:
# # Save the model
# model.save('nasnet-mobile.h5')

### MobileNetV2

In [ ]:
# from tensorflow.keras.layers import Input
# from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# # 1. Create model
# input_shape=(224, 224, 3)
# img_input = Input(shape=input_shape)

# base_model = MobileNetV2(include_top=False, input_tensor=img_input, input_shape=input_shape, 
#                          pooling="avg", weights='imagenet')
# x = base_model.output
# predictions = Dense(len(labels), activation="sigmoid", name="output")(x)
# model = Model(inputs=img_input, outputs=predictions)

# # 2. Compile the model
# optimizer = Adam(lr=0.0005)
# model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()])

# earlystopper = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=3, verbose=0, mode='min',
#     restore_best_weights=True
# )

# # 3. Fit the model
# history = model.fit(train_data,
#                    epochs=5,
#                    steps_per_epoch=len(train_data),
#                    validation_data=valid_data,
#                    validation_steps=len(valid_data),
#                    callbacks=[earlystopper])

In [ ]:
# # Save the model
# model.save('mobilenetv2.h5')

### ResNet50V2

#### Without rescaling dataset

In [ ]:
# train_datagen = ImageDataGenerator()
# valid_datagen = ImageDataGenerator()
# test_datagen = ImageDataGenerator()

In [ ]:
# IMG_SHAPE = (224, 224)
# BATCH_SIZE = 32

# print("Training images:")
# train_data = train_datagen.flow_from_dataframe(X_train,
#                                              x_col="path_gambar",
#                                              y_col=labels,
#                                              target_size=IMG_SHAPE,
#                                              classes=labels,
#                                              color_mode="rgb",
#                                              class_mode="raw",
#                                              seed=42,
#                                              batch_size=BATCH_SIZE)
# print("Validation images:")
# valid_data = valid_datagen.flow_from_dataframe(X_test,
#                                              x_col="path_gambar",
#                                              y_col=labels,
#                                              target_size=IMG_SHAPE,
#                                              classes=labels,
#                                              color_mode="rgb",
#                                              class_mode="raw",
#                                              seed=42,
#                                              batch_size=BATCH_SIZE)
# print("Testing images:")
# test_data = test_datagen.flow_from_dataframe(test,
#                                              x_col="path_gambar",
#                                              y_col=labels,
#                                              target_size=IMG_SHAPE,
#                                              classes=labels,
#                                              color_mode="rgb",
#                                              class_mode="raw",
#                                              shuffle=False,
#                                              seed=42,
#                                              batch_size=BATCH_SIZE)

In [ ]:
# from tensorflow.keras.layers import Input
# from tensorflow.keras.applications.resnet_v2 import ResNet50V2
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# # 1. Create model
# input_shape=(224, 224, 3)
# img_input = Input(shape=input_shape)

# base_model = ResNet50V2(include_top=False, input_tensor=img_input, input_shape=input_shape, 
#                          pooling="avg", weights='imagenet')
# x = base_model.output
# predictions = Dense(len(labels), activation="sigmoid", name="output")(x)
# model = Model(inputs=img_input, outputs=predictions)

# # 2. Compile the model
# optimizer = Adam(lr=0.0005)
# model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()])

# earlystopper = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=3, verbose=0, mode='min',
#     restore_best_weights=True
# )

# # 3. Fit the model
# history = model.fit(train_data,
#                    epochs=5,
#                    steps_per_epoch=len(train_data),
#                    validation_data=valid_data,
#                    validation_steps=len(valid_data),
#                    callbacks=[earlystopper])

In [ ]:
# # Save the model
# model.save('resnet50v2.h5')

### EfficientNetB0 

#### Without rescaling dataset

In [ ]:
# train_datagen = ImageDataGenerator()
# valid_datagen = ImageDataGenerator()
# test_datagen = ImageDataGenerator()

In [ ]:
# IMG_SHAPE = (224, 224)
# BATCH_SIZE = 32

# print("Training images:")
# train_data = train_datagen.flow_from_dataframe(X_train,
#                                              x_col="path_gambar",
#                                              y_col=labels,
#                                              target_size=IMG_SHAPE,
#                                              classes=labels,
#                                              color_mode="rgb",
#                                              class_mode="raw",
#                                              seed=42,
#                                              batch_size=BATCH_SIZE)
# print("Validation images:")
# valid_data = valid_datagen.flow_from_dataframe(X_test,
#                                              x_col="path_gambar",
#                                              y_col=labels,
#                                              target_size=IMG_SHAPE,
#                                              classes=labels,
#                                              color_mode="rgb",
#                                              class_mode="raw",
#                                              seed=42,
#                                              batch_size=BATCH_SIZE)
# print("Testing images:")
# test_data = test_datagen.flow_from_dataframe(test,
#                                              x_col="path_gambar",
#                                              y_col=labels,
#                                              target_size=IMG_SHAPE,
#                                              classes=labels,
#                                              color_mode="rgb",
#                                              class_mode="raw",
#                                              shuffle=False,
#                                              seed=42,
#                                              batch_size=BATCH_SIZE)

In [ ]:
# from tensorflow.keras.layers import Input
# from tensorflow.keras.applications.efficientnet import EfficientNetB0 
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# # 1. Create model
# input_shape=(224, 224, 3)
# img_input = Input(shape=input_shape)

# base_model = EfficientNetB0(include_top=False, input_tensor=img_input, input_shape=input_shape, 
#                          pooling="avg", weights='imagenet')
# x = base_model.output
# predictions = Dense(len(labels), activation="sigmoid", name="output")(x)
# model = Model(inputs=img_input, outputs=predictions)

# # 2. Compile the model
# optimizer = Adam(lr=0.0005)
# model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()])

# earlystopper = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=3, verbose=0, mode='min',
#     restore_best_weights=True
# )

# # 3. Fit the model
# history = model.fit(train_data,
#                    epochs=5,
#                    steps_per_epoch=len(train_data),
#                    validation_data=valid_data,
#                    validation_steps=len(valid_data),
#                    callbacks=[earlystopper])

In [ ]:
# Save the model
# model.save('efficientnetb0.h5')

### InceptionV3

* Khusus untuk InceptionV3, model ini memerluhkan size image `(299, 299)`

In [ ]:
# Memuat dataloader menggunakan flow_from_dataframe karena data kami terdapat di dataframe 
# IMG_SHAPE = (299, 299)
# BATCH_SIZE = 32

# print("Training images:")
# train_data = train_datagen.flow_from_dataframe(X_df,
#                                              x_col="path_gambar",
#                                              y_col=labels,
#                                              target_size=IMG_SHAPE,
#                                              classes=labels,
#                                              color_mode="rgb",
#                                              class_mode="raw",
#                                              seed=42,
#                                              batch_size=BATCH_SIZE)
# print("Validation images:")
# valid_data = valid_datagen.flow_from_dataframe(Y_df,
#                                              x_col="path_gambar",
#                                              y_col=labels,
#                                              target_size=IMG_SHAPE,
#                                              classes=labels,
#                                              color_mode="rgb",
#                                              class_mode="raw",
#                                              seed=42,
#                                              batch_size=BATCH_SIZE)
# print("Testing images:")
# test_data = test_datagen.flow_from_dataframe(test,
#                                              x_col="path_gambar",
#                                              y_col=labels,
#                                              target_size=IMG_SHAPE,
#                                              classes=labels,
#                                              color_mode="rgb",
#                                              class_mode="raw",
#                                              shuffle=False,
#                                              seed=42,
#                                              batch_size=BATCH_SIZE)

In [ ]:
# from tensorflow.keras.layers import Input
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# # 1. Create model
# input_shape=(299, 299, 3)
# img_input = Input(shape=input_shape)

# base_model = InceptionV3(include_top=False, input_tensor=img_input, input_shape=input_shape, 
#                          pooling="avg", weights='imagenet')
# x = base_model.output
# predictions = Dense(len(labels), activation="sigmoid", name="output")(x)
# model = Model(inputs=img_input, outputs=predictions)


# # 2. Compile the model
# optimizer = Adam(lr=0.0005)
# model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy(), 
#                                                                         tf.keras.metrics.AUC()])

# earlystopper = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=2, verbose=0, mode='min',
#     restore_best_weights=True
# )

# # 3. Fit the model
# history = model.fit(train_data,
#                    epochs=5,
#                    steps_per_epoch=len(train_data),
#                    validation_data=valid_data,
#                    validation_steps=len(valid_data),
#                    callbacks=[earlystopper])

In [ ]:
# model.save("inception_v3.h5")

### DenseNet121

In [ ]:
# from tensorflow.keras.layers import Input
# from tensorflow.keras.applications.densenet import DenseNet121
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# # 1. Create model
# input_shape=(224, 224, 3)
# img_input = Input(shape=input_shape)

# base_model = DenseNet121(include_top=False, input_tensor=img_input, input_shape=input_shape, 
#                          pooling="avg", weights='imagenet')
# x = base_model.output
# predictions = Dense(len(labels), activation="sigmoid", name="output")(x)
# model = Model(inputs=img_input, outputs=predictions)

# # 2. Compile the model
# optimizer = Adam(lr=0.0005)
# model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy(), 
#                                                                         tf.keras.metrics.AUC(),
#                                                                         tf.keras.metrics.Precision(),
#                                                                         tf.keras.metrics.Recall()])

# earlystopper = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=10, verbose=0, mode='min',
#     restore_best_weights=True
# )




# # 3. Fit the model
# history = model.fit(train_data,
#                    epochs=5,
#                    steps_per_epoch=len(train_data),
#                    validation_data=valid_data,
#                    validation_steps=len(valid_data),
#                    callbacks=[earlystopper])

In [ ]:
# Save the model
# model.save('densenet121.h5')

## 6. Testing dan evaluasi model

In [ ]:
test_data # pastikan testing dataset sudah ready

### Load dan predict

Beberapa model diatas sudah dilatih dan siap untuk diload dan predict

In [ ]:
vgg19 = tf.keras.models.load_model("../input/nih-chest-x-ray-models/vgg19.h5")
vgg19_pred = vgg19.predict(test_data, steps=len(test_data), verbose=1)

In [ ]:
ds_121 = tf.keras.models.load_model("../input/nih-chest-x-ray-models/densenet121.h5")
ds_121_pred = ds_121.predict(test_data, steps=len(test_data), verbose=1)

In [ ]:
nasnet = tf.keras.models.load_model("../input/nih-chest-x-ray-models/nasnet-mobile.h5")
nasnet_pred = nasnet.predict(test_data, steps=len(test_data), verbose=1)

In [ ]:
inception_v3 = tf.keras.models.load_model("../input/nih-chest-x-ray-models/inception_v3.h5")
inceptionv3_pred = inception_v3.predict(test_data, steps=len(test_data), verbose=1)

In [ ]:
mobile_v2 = tf.keras.models.load_model("../input/nih-chest-x-ray-models/mobilenetv2.h5")
mobile_v2_pred = mobile_v2.predict(test_data, steps=len(test_data), verbose=1)

### No rescaling models

In [ ]:
test_datagen = ImageDataGenerator()

print("Testing images:")
test_data = test_datagen.flow_from_dataframe(test,
                                             x_col="path_gambar",
                                             y_col=labels,
                                             target_size=IMG_SHAPE,
                                             classes=labels,
                                             color_mode="rgb",
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=42,
                                             batch_size=BATCH_SIZE)

In [ ]:
effb0 = tf.keras.models.load_model("../input/nih-chest-x-ray-models/efficientnetb0.h5")
effb0_pred = effb0.predict(test_data, steps=len(test_data), verbose=1)

In [ ]:
res50v2 = tf.keras.models.load_model("../input/nih-chest-x-ray-models/resnet50v2.h5")
res50v2_pred = res50v2.predict(test_data, steps=len(test_data), verbose=1)

#### Visualisasi dari dataset yang sudah diprediksi

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Testing images:")
test_data = test_datagen.flow_from_dataframe(test,
                                             x_col="path_gambar",
                                             y_col=labels,
                                             target_size=IMG_SHAPE,
                                             classes=labels,
                                             color_mode="rgb",
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=42,
                                             batch_size=BATCH_SIZE)

In [ ]:
test_data.reset()
test_x, test_y = next(test_data)  
print(f"Diseases vector: {test_y[22]}")
print(f"Prediction product vector: {ds_121_pred[22]*100}") # melihat prediksi dari model DenseNet121

In [ ]:
len(test_x[1, :, :, 0]) # image size, 1 -> batch

### Melihat prediksi model

1. Melakukan looping untuk setiap plot dari figure matplotlib
2. Load image dari index batch pertama pada test dataset
3. Mengambil label penyakit sebenarnya dari gambar tersebut
4. Mengambil label penyakit yang diprediksi dari gambar tersebut (ini termasuk melihat persentase label penyakit aslinya, dan jenis penyakit lain yang diatas 50%)
5. Plot gambar serta judul yang berisi label penyakit aslinya serta label penyakit yang diprediksi

In [ ]:
def plot_predictions(model_name, y_pred):
    sickest_idx = np.argsort(np.sum(test_y, axis=1) < 1)
    # 1. Setiap baris dari actual label akan dijumlahkan, apabila jumlah hasilnya 0 maka tidak ada penyakit -> "No Finding"
    # 2. Apabila terdapat penyakit / jumlah baris labelnya > 0, maka ditandakan dengan False, dan sebaliknya untuk yang No Finding dengan True
    # 3. Melakukan index sorting untuk mengurutkan dataset dari yang ada penyakit hingga No Finding

    fig, axs = plt.subplots(3, 3, figsize = (20, 20))
    fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

    for (idx, ax) in zip(sickest_idx, axs.flatten()):
        img = test_x[idx, :, :, 0] # get image
        ax.imshow(img) 

        actual_label = [label[:4] for label, value in zip(labels, test_y[idx]) if value == 1] # zip prediction labels dan actual labels

        predict_label = [f'{disease[:4]}:{pred_logit*100:.2f}%' for disease, label, pred_logit in zip(labels, test_y[idx], y_pred[idx]) if (label == 1) or (pred_logit > 0.5)]

        ax.set_title(f'Actual: {", ".join(actual_label)}\nPredict: {", ".join(predict_label)}')
        plt.figtext(0.5, 0.001, 'img_size=(224, 224) | loss=binary | batch_size=32 | epochs=5 | optimizer=adam | learning_rate=0.0005 | 10 labels | *img_size_inception_v3=(299, 299)', horizontalalignment='center', fontstyle="italic")
        ax.axis('off')
    plt.suptitle(model_name, fontsize=24, fontweight="bold", x="0.5", y="1.05")

In [ ]:
plot_predictions("ResNet50V2 predictions", res50v2_pred)

In [ ]:
plot_predictions("VGG19 predictions", vgg19_pred)

In [ ]:
plot_predictions("DenseNet121 predictions", ds_121_pred)

In [ ]:
plot_predictions("EfficientNetB0 predictions", effb0_pred)

In [ ]:
plot_predictions("InceptionV3 predictions", inceptionv3_pred)

In [ ]:
plot_predictions("NASNet-Mobile predictions", nasnet_pred)

In [ ]:
plot_predictions("MobileNetV2 predictions", mobile_v2_pred)

### Melihat ROC curve dari beberapa model

Kami menggunakan metrik auc untuk mengukur `false posivitve rate` dan `true positive rate` dari setiap label.

Plot hasil ROC curve untuk setiap label jenis penyakit dari beberapa model yang sudah diload

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
def get_auc_score(pred):
    auc = roc_auc_score(test_data.labels, pred)
    return auc

In [ ]:
plt.figure(figsize=(30, 20))
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

# DenseNet121
plt.subplot(331)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data.labels[:, idx].astype(int), ds_121_pred[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"DenseNet121 (avg: {get_auc_score(ds_121_pred):.2f})", fontsize=18)

# NASNet-Mobile
plt.subplot(332)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data.labels[:, idx].astype(int), nasnet_pred[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"NASNet-Mobile (avg: {get_auc_score(nasnet_pred):.2f})", fontsize=18)

# EffB0
plt.subplot(333)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data.labels[:, idx].astype(int), effb0_pred[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"EfficientNetB0 (avg: {get_auc_score(effb0_pred):.2f})", fontsize=18)

# VGG19
plt.subplot(334)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data.labels[:, idx].astype(int), vgg19_pred[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"VGG19 (avg: {get_auc_score(vgg19_pred):.2f})", fontsize=18)

# InceptionV3
plt.subplot(335)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data.labels[:, idx].astype(int), inceptionv3_pred[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"InceptionV3 (avg: {get_auc_score(inceptionv3_pred):.2f})", fontsize=18)

# ResNet50V2
plt.subplot(336)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data.labels[:, idx].astype(int), res50v2_pred[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"ResNet50V2 (avg: {get_auc_score(res50v2_pred):.2f})", fontsize=18)

# MobileNetV2
plt.subplot(338)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data.labels[:, idx].astype(int), mobile_v2_pred[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"MobileNetV2 (avg: {get_auc_score(mobile_v2_pred):.2f})", fontsize=18)

plt.figtext(0.5, 0.07, 'img_size=(224, 224) | loss=binary | batch_size=32 | epochs=5 | optimizer=adam | learning_rate=0.0005 | 10 labels | *img_size_inception_v3=(299, 299)', horizontalalignment='center', fontstyle="italic")
plt.suptitle("Skor AUC seluruh model pada testing dataset", fontsize=24, x="0.5", y="0.92", fontweight="bold")
plt.show()

In [ ]:
auc_score_results = pd.DataFrame([get_auc_score(ds_121_pred), get_auc_score(vgg19_pred), get_auc_score(nasnet_pred), get_auc_score(res50v2_pred), get_auc_score(inceptionv3_pred), get_auc_score(effb0_pred), get_auc_score(mobile_v2_pred)], columns=["auc_score"], index=["DenseNet121", "VGG19", "NASNet-Mobile", "ResNet50V2", "InceptionV3", "EfficientNetB0", "MobileNetV2"]).sort_values(by=['auc_score'], ascending=False)
auc_score_results.auc_score = (auc_score_results.auc_score*100).astype(int)

# Bar Plot
plt.figure(figsize=(15, 7))
ax = sns.barplot(y=auc_score_results.index, x=auc_score_results.auc_score, palette="icefire");
for container in ax.containers:
    ax.bar_label(container, fontsize="11")
    
plt.figtext(0.5, 0.001, 'img_size=(224, 224) | loss=binary | batch_size=32 | epochs=5 | optimizer=adam | learning_rate=0.0005 | 10 labels | *img_size_inception_v3=(299, 299)', horizontalalignment='center', fontstyle="italic", fontsize="10")
plt.title("Rata-rata skor AUC pada testing dataset", fontsize=15, fontweight="bold")
plt.xlabel("auc_score")
plt.ylabel("models");

Dapat disimpulkan bahwa DenseNet121 lebih unggul daripada yang lain